In [ ]:
#modules
from rxn4chemistry import RXN4ChemistryWrapper #ibm RXN API
from rdkit import Chem
from rdkit.Chem import AllChem
from IPython.display import display as dp
import time
from time import sleep
import requests
import os
from rdkit.Chem import rdChemReactions as Reactions
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys

In [ ]:
def collect_reactions(tree):
        reactions = []
        if 'children' in tree and len(tree['children']):
            reactions.append(
                AllChem.ReactionFromSmarts('{}>>{}'.format(
                    '.'.join([node['smiles'] for node in tree['children']]),tree['smiles']),useSmiles = True)
            )
        for node in tree['children']:
            reactions.extend(collect_reactions(node))
        return reactions
    
def get_reacSMILES(reac):
    reac_smiles = [cp.resolve]
    return '.'.join(smiles)

def progress_report(step):
    print(time.asctime(time.localtime(time.time())) + ' : ' + step)
    
def run_ibm_rxn(rxn_index):
    project = prodSMILES[rxn_index]
    reactants = synth_reacSMILES[rxn_index]
    rxn4chemistry_wrapper = RXN4ChemistryWrapper(api_key = #insert api key)
    progress_report("Logged in." )
    response = rxn4chemistry_wrapper.create_project(project)
    progress_report("project '" + project + "' created.")
    progress_report("Synthesis begun.")
    sleep(2)
    synth_response = rxn4chemistry_wrapper.predict_reaction(reactants)
    sleep(2)
    while True:#close loop only when synthesis is complete
        #run synthesis
        synth_results = rxn4chemistry_wrapper.get_predict_reaction_results(synth_response['prediction_id'])
        #pause for 2 sec to avoid overloading IBM-RXN
        sleep(2)
        #if synthesis is complete,...
        if synth_results['response']['payload']['status'] == 'SUCCESS':
            sleep(60)
            progress_report("Synthesis Complete!")
            break
        else:
            progress_report('Synthesis Incomplete')
            synth_results = rxn4chemistry_wrapper.get_predict_reaction_results(synth_response['prediction_id'])
            sleep(60)
    synth_output_rxnSMILES = [synth_results['response']['payload']['attempts'][i]['smiles'] for i in range(len(synth_results['response']['payload']['attempts']))]
    synth_prod_mols = [Chem.MolFromSmiles(smiles) for smiles in [project,synth_output_rxnSMILES[0][synth_output_rxnSMILES[0].index('>')+2:]]]
    synth_prod_tanimoto = DataStructs.TanimotoSimilarity(MACCSkeys.GenMACCSKeys(synth_prod_mols[0]),MACCSkeys.GenMACCSKeys(synth_prod_mols[1]))
    print('Tanimoto Similarity : '+ str(synth_prod_tanimoto))
    for i in range(len(synth_output_rxnSMILES)):
        if prodSMILES[rxn_index] == synth_output_rxnSMILES[i][synth_output_rxnSMILES[i].index('>')+2:]:
            predicted_path = synth_output_rxnSMILES
            product = prodSMILES[rxn_index]
            progress_report('Product SMILES found in synthetic paths!\n Continuing to retrosynthesis.')
            reactants = []
            progress_report('Retrosynthesis begun.')
            retro_response = rxn4chemistry_wrapper.predict_automatic_retrosynthesis(product = product)
            sleep(2)
            while True:#close loop only when retrosynthesis is complete
                #run retrosynthesis
                retro_results = rxn4chemistry_wrapper.get_predict_automatic_retrosynthesis_results(retro_response['prediction_id'])
                #pause for 2 sec to avoid overloading IBM-RXN
                sleep(2)
                #if retrosynthesis is complete,...
                if retro_results['status'] == 'SUCCESS':
                    progress_report("Retrosynthesis Complete!")
                    for index, path in enumerate(retro_results['retrosynthetic_paths']):
                        print('Showing path {} with confidence {}:'.format(index + 1, path['confidence']))
                        for reaction in collect_reactions(path): 
                            display(Chem.Draw.ReactionToImage(reaction))
                            reactants.append(path['children'][0])
                    break
                else:
                    progress_report('Retrosynthesis Incomplete')
                    retro_results = rxn4chemistry_wrapper.get_predict_automatic_retrosynthesis_results(retro_response['prediction_id'])
                    sleep(60)
            children = [retro_results['retrosynthetic_paths'][i]['children'] for i in range(len(retro_results['retrosynthetic_paths']))]
            reactants = [[]] * len(children)
            for i in range(len(reactants)):
                reactants[i] = [reactant['smiles'] for reactant in children[i]]
            reacSMILES = ['.'.join(reactantlist) for reactantlist in reactants] 
            for smiles in reacSMILES:
                if smiles == synth_reacSMILES:
                    progress_report("Synthesis matches Retrosynthesis!\n Writing reaction SMILES to 'success-DA.txt'")
                    with open ('/Users/nicholasdigrandi/compchem_proj/success-DA.txt','a') as file:
                        file.write(synth_rxnSMILES[0] + '\n')
                else:
                    progress_report("No retrosynthetic matches. \n Writing reaction SMILES to 'failed-DA.txt'")
                    with open('/Users/nicholasdigrandi/compchem_proj/failed-DA.txt','a') as file:
                        file.write(synth_rxnSMILES[0] + '\n')
        else:
            progress_report('Product SMILES NOT found in synthetic paths.\n Printing synthetic paths.')
            for path in synth_output_rxnSMILES:
                rxn = Reactions.ReactionFromSmarts(path, useSmiles = True)
                display(Chem.Draw.ReactionToImage(rxn))
            reactants = []
            progress_report('Retrosynthesis begun.')
            retro_response = rxn4chemistry_wrapper.predict_automatic_retrosynthesis(product = prodSMILES[rxn_index])
            sleep(2)
            while True:#close loop only when retrosynthesis is complete
                #run retrosynthesis
                retro_results = rxn4chemistry_wrapper.get_predict_automatic_retrosynthesis_results(retro_response['prediction_id'])
                #pause for 2 sec to avoid overloading IBM-RXN
                sleep(2)
                #if retrosynthesis is complete,...
                if retro_results['status'] == 'SUCCESS':
                    progress_report("Retrosynthesis Complete!")
                    for index, path in enumerate(retro_results['retrosynthetic_paths']):
                        print('Showing path {} with confidence {}:'.format(index + 1, path['confidence']))
                        for reaction in collect_reactions(path): 
                            display(Chem.Draw.ReactionToImage(reaction))
                            reactants.append(path['children'][0])
                    break
                else:
                    progress_report('Retrosynthesis Incomplete')
                    retro_results = rxn4chemistry_wrapper.get_predict_automatic_retrosynthesis_results(retro_response['prediction_id'])
                    sleep(60)
            children = [retro_results['retrosynthetic_paths'][i]['children'] for i in range(len(retro_results['retrosynthetic_paths']))]
            reactants = [[]] * len(children)
            for i in range(len(reactants)):
                reactants[i] = [reactant['smiles'] for reactant in children[i]]
            reacSMILES = ['.'.join(reactantlist) for reactantlist in reactants]     
            for smiles in reacSMILES:
                if smiles == synth_reacSMILES:
                    progress_report("Synthesis matches Retrosynthesis!\n Writing reaction SMILES to 'success-DA.txt'")
                    with open ('/Users/nicholasdigrandi/compchem_proj/success-DA.txt','a') as file:
                        file.write(synth_rxnSMILES[0] + '\n')
                else:
                    progress_report("No retrosynthetic matches. \n Writing reaction SMILES to 'failed-DA.txt'")
                    with open('/Users/nicholasdigrandi/compchem_proj/failed-DA.txt','a') as file:
                        file.write(synth_rxnSMILES[0] + '\n')